# Habla imaginada - Clasificación de señales de EEG | Agustín Marzioni

##Preprocesamiento

Primero importo ambos dataframes de entrenamiento

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd

X_train = pd.read_csv("/content/gdrive/My Drive/IA@Litoral 2023/Competencia/Resultados participantes/IAlitoral/habla-imaginada-clasificacion-senales-EEG (1)/X_train/X_train")
# Acá estoy importando el dataset
y_train = pd.read_csv("/content/gdrive/My Drive/IA@Litoral 2023/Competencia/Resultados participantes/IAlitoral/habla-imaginada-clasificacion-senales-EEG (1)/y_train.csv")

Voy a reordenar los datos en un diccionario que en cada key contenga un numpy array de tres dimensiones con los datos de una persona (la primer será el número de sesiones de cada individuo, la segunda el número de canales y la tercera el número de instantes). Hago esto para facilitar el futuro proceso de entrenamiento

In [ ]:

# creo otro df solamente para recortar más facilmente los nombres de los canales e instantes
serie = pd.DataFrame(X_train.columns)
serie.rename(columns={0: "columnas crudas"}, inplace=True)
serie['Canal'], serie['Instante'] = serie["columnas crudas"].str.split('_', 1).str
del serie["columnas crudas"]


# reordeno las filas del df segun la columna sujeto, asi me quedan todos los sujetos separados
X_train["label"] = y_train["Data"].values
X_train = X_train.sort_values("sujeto")
X_train.reset_index(inplace=True)
y_train["Data"] = X_train["label"]
del X_train["index"]
del X_train["label"]
df_personas = pd.DataFrame(X_train["sujeto"])
X_train = X_train.T

X_train["Canal"] = serie["Canal"].values
X_train["Instante"] = serie["Instante"].values
# borro la ultima fila, en donde está la etiqueta "Sujeto"
X_train = X_train.dropna()
# pongo un indice jerarquico
X_train = X_train.set_index(["Canal", "Instante"])


<ipython-input-14-2f91d4199656>:4: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  serie['Canal'], serie['Instante'] = serie["columnas crudas"].str.split('_', 1).str
<ipython-input-14-2f91d4199656>:4: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  serie['Canal'], serie['Instante'] = serie["columnas crudas"].str.split('_', 1).str


In [ ]:
def crear_xdict(cota_inf,cota_sup,X_train,df_personas):
    import numpy as np


    # lista con los nombres únicos de las personas
    personas = df_personas["sujeto"].unique()

    # el número de trials POR PERSONA, canales e instantes
    num_trials = df_personas['sujeto'].value_counts()
    num_trials.sort_index(inplace=True)
    num_canales = X_train.index.get_level_values('Canal').unique().shape[0]
    num_instantes = X_train.index.get_level_values('Instante').unique().shape[0]
    # Voy a recortar la información para quedarme solo con la data desde el cota_inf hasta el cota_sup


    num_instantes = num_instantes - cota_inf - cota_sup
    # Voy a poner los datos de todas las personas en un diccionario, donde las keys son los nombres de la persona
    eeg_dict = {}
    trial_vistos = 0
    for i, persona in enumerate(personas):

        eeg_array_persona = np.empty((num_trials[i], num_canales, num_instantes))

        for trial in range(num_trials[i]):
            instantes_vistos = cota_inf
            for canal in range(num_canales):
                # Tengo una columna por cada "sesión" o "trial"
                eeg_array_persona[trial,canal,:] = X_train.iloc[instantes_vistos : instantes_vistos + num_instantes][trial_vistos]

                instantes_vistos = instantes_vistos + num_instantes + cota_inf + cota_sup

            trial_vistos = trial_vistos + 1

        eeg_dict[persona] = eeg_array_persona
    return eeg_dict

In [ ]:
# Ahora hago exactamente lo mismo pero con los y_labels
def crear_ydict(y_train,df_personas):
    y_dict = {}

    personas = df_personas["sujeto"].unique()

    # el número de trials POR PERSONA, canales e instantes
    num_trials = df_personas['sujeto'].value_counts()
    num_trials.sort_index(inplace=True)

    y_vistos = 0
    for i, persona in enumerate(personas):

        resultados_persona = np.empty((num_trials[i]))


        resultados_persona[:] = y_train.iloc[y_vistos : y_vistos+ num_trials[i], 0]

        y_vistos = y_vistos + num_trials[i]
        y_dict[persona] = resultados_persona

    return y_dict


Ahora voy a crear una funcion para filtrar todos los numpy arrays creados, borrando todas las frecuencia por encima de 40. para aumentar la proporción señal/ruido

In [ ]:
from scipy import signal
def filtrar(eeg_dict):
    # Definir las frecuencias de corte y el orden del filtro
    freqs = [1., 40.]
    order = 4
    sfreq = 1024

    # Crear el filtro
    b, a = signal.butter(order, freqs, 'bandpass', fs=sfreq)

    # Aplicar el filtro a los datos de EEG
    for persona in eeg_dict:
        # Obtener los datos de EEG para esta persona
        eeg_array_persona = eeg_dict[persona]

        # Filtrar los datos
        for trial in range(eeg_array_persona.shape[0]):
            for canal in range(eeg_array_persona.shape[1]):
                eeg_array_persona[trial, canal, :] = signal.filtfilt(b, a, eeg_array_persona[trial, canal, :])

        # Guardar los datos filtrados de nuevo en el diccionario
        eeg_dict[persona] = eeg_array_persona
    return eeg_dict


Crearé el diccionario con los instantes a partir del milisegundo 300, recortando la onda P300 y favoreciendo la proporción señal/ruido

In [ ]:
cota_inf = 300
cota_sup = 0
eeg_dict = crear_xdict(cota_inf,cota_sup,X_train,df_personas)

import numpy as np
y_dict = crear_ydict(y_train, df_personas)


In [ ]:
eeg_dict = filtrar(eeg_dict)


En este punto dividí mi trabajo en varias ramas, probando distintos modelos para hacer predicciones. En un inicio, extraje el DWT en 6 niveles de la señal con daubechies y calculé 7 métricas por cada onda: máximo, mínimo, media, std, energía, entropía y el poder medio. Estas métricas demostraron ser útiles en Abukhettala, K., & Ata, O. (2022). Con estos datos entrené modelos de CNN (62%), RF (57%), Naive Bayes (57%) y SVM (58%).

No los desarrollaré a continuación porque el modelo que terminó mostrando mayor accuracy fue EEGNetv4 de la biblioteca Braindecode. Este modelo fue diseñado para tratar datos de eeg sin un previo trabajo de extracción de características.

### Extracción de características (no utilizado)
Por si interesa, había calculado el DWT y sus métricas de la siguiente forma (aunque no las utlicé en el modelo final):

In [ ]:
import pywt
import numpy as np
import scipy.stats

# Aplicar la transformada de wavelet a los datos de EEG
for persona in eeg_dict:
    # Obtener los datos de EEG para esta persona
    eeg_array_persona = eeg_dict[persona]

    # Crear un nuevo diccionario para almacenar las estadísticas de los coeficientes de wavelet
    wavelet_stats = {}

    # Descomponer los datos
    for trial in range(eeg_array_persona.shape[0]):
        for canal in range(eeg_array_persona.shape[1]):
            # Aplicar la transformada de wavelet
            coeffs = pywt.wavedec(eeg_array_persona[trial, canal, :], 'db4', level=6)

            # Calcular las estadísticas para cada sub-banda
            for i, coeff in enumerate(coeffs):
                # Crear una clave para esta sub-banda si no existe
                if i not in wavelet_stats:
                    wavelet_stats[i] = {'max': [], 'min': [], 'mean': [], 'std': [], 'entropy': [], 'energy': [], 'medium_power': []}

                # Calcular las estadísticas y añadirlas a la lista
                wavelet_stats[i]['max'].append(np.max(coeff))
                wavelet_stats[i]['min'].append(np.min(coeff))
                wavelet_stats[i]['mean'].append(np.mean(coeff))
                wavelet_stats[i]['std'].append(np.std(coeff))
                wavelet_stats[i]['entropy'].append(scipy.stats.entropy(coeff))
                wavelet_stats[i]['energy'].append(np.sum(coeff**2))
                wavelet_stats[i]['medium_power'].append(np.median(coeff**2))


    # Guardar las estadísticas de los coeficientes de wavelet de vuelta en el diccionario
    # eeg_dict[persona] = wavelet_stats

## Modelo EEGNetv4

Iniciando con la implementacion del modelo EEGNetv4, descargué los paquetes necesarios

In [ ]:
!pip install moabb

!pip install braindecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.0/365.0 kB 29.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyriemann: filename=pyriemann-0.3-py2.py3-none-any.whl size=78031 sha256=43eec1373ca48f941f6db724d1b0a025a0a43938653ebaf35ac94f14e8986009
  Stored in directory: /root/.cache/pip/wheels/eb/52/63/ad042f5ca1209b213a326a843e75d730b30bc7a89a79edb187
Successfully built pyriemann
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 8.6 MB/s eta 0:00:00


### Búsqueda de hiperparámetros
Busqué el batchsize y learning rate óptimos entrando el modelo por 10 épocas (no utilicé un mayor numero de épocas para no incrementar el tiempo de la búsqueda, no tengo gpu en mi computadora y tampoco créditos en google colab):

In [ ]:
from braindecode.models import EEGNetv4
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
from braindecode import EEGClassifier
import torch


model = EEGNetv4(6, 2, 4096-cota_inf-cota_sup)

# Definir los parámetros para la búsqueda en grid
parameters = {'lr': [0.01, 0.001, 0.0001], 'batch_size': [16, 32, 64]}

# Crear el clasificador EEG
clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=None,
    iterator_train=DataLoader,
    iterator_valid=DataLoader,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# Crear el objeto GridSearchCV
gs = GridSearchCV(clf, parameters)

# Realizar la validación cruzada manualmente
keys = list(eeg_dict.keys())
for i in range(len(keys)):
    test_keys = keys[i]
    train_keys = keys[:i] + keys[i+1:]

    # Crear los conjuntos de datos de entrenamiento y prueba
    X_train_list = [eeg_dict[key] for key in train_keys]
    y_train_list = [y_dict[key] for key in train_keys]
    X_test_list = [eeg_dict[key] for key in test_keys]
    y_test_list = [y_dict[key] for key in test_keys]

    # Concatenar los conjuntos de datos de entrenamiento y prueba
    X_train_concat = np.concatenate(X_train_list)
    y_train_concat = np.concatenate(y_train_list)
    X_test_concat = np.concatenate(X_test_list)
    y_test_concat = np.concatenate(y_test_list)

    # Ajustar el modelo a los datos de entrenamiento y buscar los mejores parámetros
    gs.fit(X_train_concat, y_train_concat)

    # Predecir los resultados del conjunto de prueba
    y_pred = gs.predict(X_test_concat)

    # Calcular e imprimir la precisión del modelo en este fold
    accuracy = accuracy_score(y_test_concat, y_pred)
    print(f'Precisión para el fold con individuo {test_keys} como prueba: {accuracy}')



# Imprimir los mejores parámetros después de la búsqueda en grid
print("Los mejores parámetros son: ", gs.best_params_)


  epoch    train_loss       dur
-------  ------------  --------
      1        1.0167  242.1017
      2        0.6104  227.8961
      3        0.5820  229.4150
      4        0.5585  227.2584
      5        0.5609  227.5960
      6        0.5583  229.3292
      7        0.5510  227.2943
      8        0.5448  227.3985
      9        0.5454  228.5626
     10        0.5479  226.9827
  epoch    train_loss       dur
-------  ------------  --------
      1        1.0207  226.3323
      2        0.6163  226.4583
      3        0.5901  228.1266
      4        0.5754  226.7894
      5        0.5819  226.9397
      6        0.5693  228.9952
      7        0.5729  226.6332
      8        0.5617  226.5017
      9        0.5748  226.7199
     10        0.5619  227.8482
  epoch    train_loss       dur
-------  ------------  --------
      1        1.0068  226.1680
      2        0.6131  226.4402
      3        0.5723  226.6553
      4        0.5640  227.0392
      5        0.5553  229.0915
      6 

En la búsqueda obtuve que el mejor batchsize era 16 y el lr 0.001.


### Entrenamiento final
Luego cargué el archivo de X_test y lo preprocesé para poder utilizarlo con mi **modelo**

In [ ]:
import pandas as pd
X_test = pd.read_csv("/content/gdrive/My Drive/IAlitoral/habla-imaginada-clasificacion-senales-EEG (1)/X_test/X_test")


In [ ]:
X_test["sujeto"].value_counts()

F    578
B    513
C    510
A    496
E    491
D    481
Name: sujeto, dtype: int64

In [ ]:
# creo otro df solamente para recortar más facilmente los canales y los instantes
del X_test["sujeto"]
serie = pd.DataFrame(X_test.columns)
serie.rename(columns={0: "columnas crudas"}, inplace=True)
serie['Canal'], serie['Instante'] = serie["columnas crudas"].str.split('_', 1).str
del serie["columnas crudas"]


# reordeno las filas del df segun la columna sujeto, asi me quedan todos los sujetos separados


X_test = X_test.T


X_test["Canal"] = serie["Canal"].values
X_test["Instante"] = serie["Instante"].values
# borro la ultima fila, en donde está la etiqueta "Sujeto"
X_test = X_test.dropna()
#pongo un indice jerarquico
X_test = X_test.set_index(["Canal", "Instante"])


<ipython-input-33-6c9c1b805ee2>:5: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  serie['Canal'], serie['Instante'] = serie["columnas crudas"].str.split('_', 1).str
<ipython-input-33-6c9c1b805ee2>:5: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  serie['Canal'], serie['Instante'] = serie["columnas crudas"].str.split('_', 1).str


In [ ]:

def crear_test(cota_inf,cota_sup,X_test):
    import numpy as np



    # el número de trials POR PERSONA, canales e instantes
    num_trials = X_test.shape[1]
    num_canales = X_test.index.get_level_values('Canal').unique().shape[0]
    num_instantes = X_test.index.get_level_values('Instante').unique().shape[0]



    num_instantes = num_instantes - cota_inf - cota_sup
    trial_vistos = 0



    ar_test = np.empty((num_trials, num_canales, num_instantes))

    for trial in range(num_trials):
            instantes_vistos = cota_inf
            for canal in range(num_canales):

                #Tengo una columna por cada trial
                ar_test[trial,canal,:] = X_test.iloc[instantes_vistos : instantes_vistos + num_instantes][trial_vistos]

                instantes_vistos = instantes_vistos + num_instantes + cota_inf + cota_sup

            trial_vistos = trial_vistos + 1
    return ar_test

In [ ]:
X_test = crear_test(cota_inf,cota_sup,X_test)

In [ ]:
from scipy import signal
import numpy as np

def filtrar(eeg_dict):
    # Definir las frecuencias de corte y el orden del filtro
    freqs = [1., 40.]
    order = 4
    sfreq = 1024

    # Crear el filtro
    b, a = signal.butter(order, freqs, 'bandpass', fs=sfreq)

    # Aplicar el filtro al array de tres dimensiones
    eeg_filtered = np.zeros_like(eeg_dict)
    for i in range(eeg_dict.shape[0]):
        for j in range(eeg_dict.shape[1]):
            eeg_filtered[i,j,:] = signal.filtfilt(b, a, eeg_dict[i,j,:])

    return eeg_filtered
X_test = filtrar(X_test)

Finalmente, entrené el modelo con todo el X_train y lo usé para predecir los valores de X_test

In [ ]:
import numpy as np
from braindecode.models import EEGNetv4
from braindecode.datautil import create_from_X_y
import torch
from torch.utils.data import DataLoader


# Crear listas para almacenar los datos y las etiquetas
data = []
labels = []

# Iterar sobre los diccionarios para llenar las listas de datos y etiquetas
for key in eeg_dict:
    data.extend(eeg_dict[key])
    labels.extend(y_dict[key])


# Convertir las listas a arrays de numpy
data = np.array(data, dtype=object)
labels = np.array(labels, dtype=object)

sfreq = 1024

train_set = create_from_X_y(data, labels, False, sfreq)

# Creo el DataLoader con un tamaño de lote de 16
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)

# Creo el modelo EEGNetv4
model = EEGNetv4(6, 2, 4096-cota_inf-cota_sup)

# Definir el optimizador y la función de pérdida
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# Entrenar el modelo, por 10 épocas
num_epochs = 10
device = 'cpu'
for epoch in range(num_epochs):  # num_epochs es el número de épocas que deseas entrenar
    for i, (inputs, labels, _) in enumerate(train_loader):

        # Convertir los inputs y labels a tensores de PyTorch
        inputs = inputs.float().to(device)
        labels = labels.long().to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


y_test = [0] * len(X_test)
test_set = create_from_X_y(X_test, y_test, False, 1024)

test_loader = DataLoader(test_set, batch_size=16)
predictions = []
for i, batch in enumerate(test_loader):
    inputs = batch[0].float().to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    predictions.extend(predicted.cpu().numpy())


/usr/local/lib/python3.10/dist-packages/braindecode/datautil/__init__.py:15: UserWarning: create_from_X_y has been moved to datasets, please use from braindecode.datasets import create_from_X_y
  warn('create_from_X_y has been moved to datasets, please use from braindecode.datasets import create_from_X_y')


Se truncaron las últimas líneas 5000 del resultado de transmisión.
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 3796 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 3796 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 3796 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 3796 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 3796 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 3796 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 3796 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 3796 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 3796 original time points ...
0 bad epochs dropped
Using data from preloaded Raw 

Por último, guardé el archivo las predicciones en un .csv

In [ ]:
import pandas as pd

# Crear un DataFrame de pandas con las predicciones y los IDs
predictions_df = pd.DataFrame({'ID': range(1, len(predictions) + 1), 'prediccion': predictions})

# Convertir las predicciones a booleanos (True para 1, False para 0)
predictions_df['prediccion'] = predictions_df['prediccion'].astype(bool)

# Guardar el DataFrame en un archivo CSV
predictions_df.to_csv('predictions.csv', index=False)

### Referencias

- Lawhern, V. J., Solon, A. J., Waytowich, N. R., Gordon, S. M., Hung, C. P., & Lance, B. J. (2018). EEGNet: A Compact Convolutional Network for EEG-based Brain-Computer Interfaces. arXiv preprint arXiv:1611.08024.

- Abukhettala, K., & Ata, O. (2022). Analyzing of EEG signals with deep learning and discrete Wavelet transform. Europan journal of science and technology. https://doi.org/10.31590/ejosat.953576